In [1]:
from time import time
from pyasn1.type import univ, char, tag, namedtype, constraint, namedval
from pyasn1.codec.ber import encoder, decoder
from trustmessages import *
from asn1ate import parser, sema, pyasn1gen
import random
import StringIO

In [2]:
SCHEMA = """QTM DEFINITIONS ::= BEGIN
QualitativeValue ::= ENUMERATED {
    very-bad (0), bad (1), neutral (2), good (3), very-good (4)
}

SL ::= SEQUENCE {
    b REAL, d REAL, u REAL
}

END"""

# parse the schema with asn1ate
_parse_tree = parser.parse_asn1(SCHEMA)
_modules = sema.build_semantic_model(_parse_tree)
_clazz_stream = StringIO.StringIO()
pyasn1gen.generate_pyasn1(_modules[0], _clazz_stream)
_clazz = _clazz_stream.getvalue()
# execute class in this context
exec(_clazz)

In [3]:
r = Assessment()
r["tms"] = univ.ObjectIdentifier((2, 2, 2))
r["source"] = "djelenc@gmail.com"
r["target"] = "david.jelenc@fri.uni-lj.si"
r["service"] = "seller"
r["date"] = time()
r["value"] = encoder.encode(QualitativeValue("very-good"))

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Assessment())
assert(r.prettyPrint() == decoded.prettyPrint())
v, _ = decoder.decode(decoded["value"], asn1Spec=QualitativeValue())

In [4]:
text = "service = seller AND source = david@fri.si AND (target = balu@fri.si OR target = aleks@fri.si)"

In [5]:
q = trustutils.create_query(text)
s = encoder.encode(q)
float(len(s)) / len(text)

0.8936170212765957

In [6]:
qtm = trustdatabase.QtmDb()
sl = trustdatabase.SLDb()

In [7]:
print trustdatabase.InMemoryTrustDatabase.pp(qtm.ASSESSMENT_DB[0])
print trustdatabase.InMemoryTrustDatabase.pp(sl.ASSESSMENT_DB[0])

Assessment:
 tms=1.1.1
 source=alice
 target=bob
 service=buyer
 date=0
 value='very-bad'
Assessment:
 tms=2.2.2
 source=alice
 target=bob
 service=buyer
 date=0
 value=<b=0.97, d=0.01, u=0.02>


In [8]:
x = trustdatabase.SLDb().TrustClass()
#x["b"] = 0.7
#x["d"] = 0.2
#x["u"] = 0.1
print x.prettyPrint()

<b=0.00, d=0.00, u=0.00>


In [9]:
import base64

In [10]:
bytes = base64.b64decode("ZlQCAQFlTwoBAGUuCgEAZAoKAQBABWRhdmlkZR0KAQFkCwoBAEMGc2VsbGVyZAsKAQBDBmxldHRlcmUaCgEBZAkKAQBBBGJhbHVkCgoBAEEFYWxla3M=")
m, _ = decoder.decode(bytes, asn1Spec=Message())
print pp(m)

Message:
 assessment-request=AssessmentRequest:
  rid=1
  query=Query:
   log=Logical:
    op='and'
    l=Query:
     log=Logical:
      op='and'
      l=Query:
       cmp=Comparison:
        op='eq'
        value=Value:
         source=david
      r=Query:
       log=Logical:
        op='or'
        l=Query:
         cmp=Comparison:
          op='eq'
          value=Value:
           service=seller
        r=Query:
         cmp=Comparison:
          op='eq'
          value=Value:
           service=letter
    r=Query:
     log=Logical:
      op='or'
      l=Query:
       cmp=Comparison:
        op='eq'
        value=Value:
         target=balu
      r=Query:
       cmp=Comparison:
        op='eq'
        value=Value:
         target=aleks


In [11]:
encode = lambda m: base64.b64encode(encoder.encode(m))
decode = lambda m: base64.b64decode(m)
to_bytes = lambda s: ":".join("{:02x}".format(ord(c)) for c in s)

In [12]:
bdu = SL()
bdu["b"] = 0.1
bdu["d"] = 0.2
bdu["u"] = 0.7

bdu2 = "MCEJCYDJDMzMzMzMzQkJgMoMzMzMzMzNCQmAzAszMzMzMzM="
decode(bdu2), encoder.encode(bdu)

('0!\t\t\x80\xc9\x0c\xcc\xcc\xcc\xcc\xcc\xcd\t\t\x80\xca\x0c\xcc\xcc\xcc\xcc\xcc\xcd\t\t\x80\xcc\x0b333333',
 '0\x15\t\x05\x031E-1\t\x05\x032E-1\t\x05\x037E-1')

In [13]:
a = Assessment()
a["tms"] = (1, 1, 1)
a["source"] = "alice"
a["target"] = "bob"
a["service"] = "seller"
a["date"] = 10
a["value"] = encoder.encode(QualitativeValue("very-bad"))
encoded = encode(a)
encoded

'aR4GAikBEwVhbGljZRMDYm9iEwZzZWxsZXICAQoKAQA='

In [14]:
v, _ = decoder.decode(base64.b64decode(encoded), asn1Spec=Assessment())
print pp(v)

Assessment:
 tms=1.1.1
 source=alice
 target=bob
 service=seller
 date=10
 value=0x0a0100


In [15]:
qtm = QualitativeValue("bad")
print to_bytes(decode("CgEB")), to_bytes(encoder.encode(qtm))

0a:01:01 0a:01:01
